In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# [R21] Line Shape - Flatte 

In [ ]:
%pip install -q ampform==0.14.8 plotly==5.17.0 sympy==1.12 tensorwaves[jax]==0.4.10

## Generate transitions

In [ ]:
from collections import defaultdict
from typing import Dict, List, Tuple
import ampform
import graphviz
import sympy as sp
from ampform import ReactionInfo
from ampform.dynamics.builder import TwoBodyKinematicVariableSet
from ampform.io import aslatex
from IPython.display import Latex, Markdown, Math
from qrules.particle import Particle
from qrules.transition import ReactionInfo
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math

In [ ]:
from __future__ import annotations

import qrules
from qrules.particle import ParticleCollection
from qrules.transition import ReactionInfo

FINAL_STATES: list[tuple[str, ...]] = [
    ["eta", "pi0", "gamma"],
    ["K0", "K~0", "gamma"],
]
REACTIONS: list[ReactionInfo] = [
    qrules.generate_transitions(
        initial_state="J/psi(1S)",
        final_state=final_state,
        # Different quantum numbers
        allowed_intermediate_particles=["a(0)(980)"],
        # Same quantum numbers
        # allowed_intermediate_particles=["N(1895)+", "N(1650)+"],
        allowed_interaction_types=["strong", "em"],
        formalism="canonical-helicity",
    )
    for final_state in FINAL_STATES
]

## Model formulation

In [ ]:
from collections import defaultdict

import ampform
import sympy as sp
from ampform.helicity import TwoBodyKinematicVariableSet
from qrules.particle import Particle

COLLECTED_X_SYMBOLS: dict[sp.Symbol, list[tuple[Particle, int]]] = defaultdict(set)


def create_dynamics_symbol(
    resonance: Particle, variable_pool: TwoBodyKinematicVariableSet
) -> Tuple[sp.Expr, Dict[sp.Symbol, float]]:
    L = sp.Rational(variable_pool.angular_momentum)
    J = sp.Rational(resonance.spin)
    Q = resonance.charge
    P = sp.Rational(resonance.parity)
    superscript = f"L={L},S={J}"
    subscript = f"Q={Q:+d},P={int(P):+d}"
    X = sp.Symbol(f"X^{{{superscript}}}_{{{subscript}}}")
    COLLECTED_X_SYMBOLS[X].add((resonance, variable_pool.angular_momentum))
    parameter_defaults = {}
    retur1n X, parameter_defaults


MODELS = []
for reaction in REACTIONS:
    builder = ampform.get_builder(reaction)
    builder.adapter.permutate_registered_topologies()
    builder.scalar_initial_state_mass = True
    builder.stable_final_state_ids = [0, 1, 2]
    for resonance in reaction.get_intermediate_particles():
        builder.set_dynamics(resonance.name, create_dynamics_symbol)
    MODELS.append(builder.formulate())

In [ ]:
from ampform.io import aslatex
from IPython.display import Math

selected_amplitudes = {
    k: v for i, (k, v) in enumerate(MODELS[0].amplitudes.items()) if i < 3
}

## K-matrix dynamics

In [ ]:
for X, resonance_info in COLLECTED_X_SYMBOLS.items():
    for res, L in sorted(resonance_info):
        display(X)
        print(f"  {res.name:<20s} {res.mass:>8g} GeV  {res.width:>8g} GeV")

In [ ]:
from dataclasses import dataclass


@dataclass
class TwoBodyDecay:  # specific to the channel
    child1: Particle
    child2: Particle


DECAYS = tuple(
    TwoBodyDecay(
        child1=reaction.final_state[0],
        child2=reaction.final_state[1],
    )
    for reaction in REACTIONS
)
s = sp.Symbol("m_01", real=True) ** 2

In [ ]:
PARAMETERS_DEFAULTS = {}
for model in MODELS:
    PARAMETERS_DEFAULTS.update(model.parameter_defaults)

In [ ]:
resonances, *_ = COLLECTED_X_SYMBOLS.values()

## Formulate $K$ matrix

In [ ]:
n_channels = len(REACTIONS)
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
rho = sp.MatrixSymbol("rho", n_channels, n_channels)

In [ ]:
from ampform.kinematics.phasespace import Kallen


@unevaluated(real=False)
class PhaseSpaceFactor_Kaellen(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return 2 * BreakupMomentum(s, m1, m2) / sp.sqrt(s)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))

In [ ]:
from ampform.dynamics import BlattWeisskopfSquared, BreakupMomentumSquared
from ampform.dynamics.builder import TwoBodyKinematicVariableSet
from ampform.dynamics.phasespace import PhaseSpaceFactor, PhaseSpaceFactorComplex
from sympy.matrices.expressions.matexpr import MatrixElement


def formulate_K_matrix(
    resonances: list[tuple[Resonances, int]], n_channels: int
) -> dict[MatrixElement, sp.Expr]:
    Kmatrix_expressions = {}
    for i in range(n_channels):
        for j in range(n_channels):
            resonance_contributions = []
            for res, L_R in resonances:
                s = sp.Symbol("m_01", evaluate=False) ** 2
                m_a_i = sp.Symbol(Rf"m_{{0,{i}}}")
                m_b_i = sp.Symbol(Rf"m_{{1,{i}}}")
                m_a_j = sp.Symbol(Rf"m_{{0,{j}}}")
                m_b_j = sp.Symbol(Rf"m_{{1,{j}}}")
                w_R = sp.Symbol(Rf"\Gamma_{{{res.latex}}}")
                gamma_Ri = sp.Symbol(Rf"\gamma_{{{res.latex},{i}}}")
                gamma_Rj = sp.Symbol(Rf"\gamma_{{{res.latex},{j}}}")
                d_R = sp.Symbol(Rf"R_{{{res.latex}}}")
                q_i = BreakupMomentum(s, m_a_i, m_b_i)
                q_j = BreakupMomentum(s, m_a_j, m_b_j)
                ff_Ri = sp.sqrt(BlattWeisskopfSquared(L_R, q_i * d_R**2))
                ff_Rj = sp.sqrt(BlattWeisskopfSquared(L_R, q_j * d_R**2))
                m_R = sp.Symbol(Rf"m_{{{res.latex}}}")
                rho_i = -ChewMandelstam(s, m_a_i, m_b_i) * 16 * sp.pi * sp.I
                rho_j = -ChewMandelstam(s, m_a_j, m_b_j) * 16 * sp.pi * sp.I
                # Default parameter values
                parameter_defaults = {
                    m_a_i: DECAYS[i].child1.mass,
                    m_b_i: DECAYS[i].child2.mass,
                    m_a_j: DECAYS[j].child1.mass,
                    m_b_j: DECAYS[j].child2.mass,
                    w_R: res.width,
                    gamma_Ri: 1,
                    gamma_Rj: 1,
                    d_R: 1,
                    m_R: res.mass,
                }
                PARAMETERS_DEFAULTS.update(parameter_defaults)
                expr = (gamma_Ri * gamma_Rj * ff_Ri * ff_Rj * m_R * w_R) / (
                    (s - m_R**2) * sp.sqrt(rho_i * rho_j)
                )
                resonance_contributions.append(expr)
            Kmatrix_expressions[K[i, j]] = sum(resonance_contributions)

    return Kmatrix_expressions


K_expressions = formulate_K_matrix(resonances, n_channels=len(REACTIONS))
Math(aslatex(K_expressions))

In [ ]:
K.as_explicit().inv()

In [ ]:
import numpy as np
import sympy as sp
from ampform.dynamics.phasespace import PhaseSpaceFactor, PhaseSpaceFactorComplex
from sympy.matrices import MatrixSymbol


def formulate_Phsp_matrix(n_channels: int) -> dict[sp.MatrixElement, sp.Expr]:
    matrix_expressions = {}

    for i in range(n_channels):
        for j in range(n_channels):
            if i == j:
                m_a_i = sp.Symbol(Rf"m_{{0,{i}}}")
                m_b_i = sp.Symbol(Rf"m_{{1,{i}}}")
                rho_i = ChewMandelstam(s, m_a_i, m_b_i) * 16 * sp.pi
                matrix_expressions[rho[i, j]] = rho_i
            else:
                matrix_expressions[rho[i, j]] = 0
                if i < len(DECAYS) and j < len(DECAYS):
                    m_a_i = sp.Symbol(Rf"m_{{0,{i}}}")
                    m_b_i = sp.Symbol(Rf"m_{{1,{i}}}")
                    rho_i = rho_i = ChewMandelstam(s, m_a_i, m_b_i) * 16 * sp.pi
                    parameter_defaults = {
                        m_a_i: DECAYS[i].child1.mass,
                        m_b_i: DECAYS[i].child2.mass,
                    }
                    PARAMETERS_DEFAULTS.update(parameter_defaults)

    return matrix_expressions


Phsp_expressions = formulate_Phsp_matrix(n_channels=len(REACTIONS))
rho.as_explicit().xreplace(Phsp_expressions).doit()

## T matrix 
In this section the $T$ matrix element $T_{00}$ for the channel $p\bar{p}\eta$ is visualized to investigate the lineshape.

In [ ]:
T = (I - K * rho).inv() * K
T_matrix = T.as_explicit()
T_matrix

## Merge dictionarys

In [ ]:
combined_expressions = {**K_expressions, **Phsp_expressions}

In [ ]:
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.interface import ParametrizedFunction


def create_function(expr: sp.Expr) -> ParametrizedFunction:
    parameter_symbols = set(expr.free_symbols) - s.free_symbols
    parameter_defaults = {
        k: v for k, v in PARAMETERS_DEFAULTS.items() if k in parameter_symbols
    }
    return create_parametrized_function(expr, parameter_defaults, backend="numpy")


T_funcs = np.array([
    [
        create_function(T_matrix[i, j].xreplace(combined_expressions).doit())
        for i in range(n_channels)
    ]
    for j in range(n_channels)
])
T_funcs[1, 1].parameters.items()

## Create widgets

In [ ]:
import ipywidgets as w

matrix_index = [i for i in range(n_channels)]
for i in range(n_channels):
    for j in range(n_channels):
        sliders = dict(
            i=w.RadioButtons(
                description=f"T matrix index {{{i}}}", options=matrix_index
            ),
            j=w.RadioButtons(
                description=f"T matrix index {{{j}}}", options=matrix_index
            ),
        )


for func in T_funcs.flatten():
    for par, value in func.parameters.items():
        value = complex(value).real
        step_ = 0.01
        if par.startswith("m_{"):
            min_ = 0.01
            max_ = 1.5
        elif par.startswith(R"\phi"):
            min_ = 0
            max_ = 2 * np.pi
            step_ = np.pi / 8
        else:
            min_ = 0
            max_ = 2 * value
        sliders[par] = w.FloatSlider(
            continuous_update=False,
            description=Rf"${par}$",
            max=max_,
            min=min_,
            step=step_,
            value=value,
        )

slider_group = w.VBox(list(sliders.values()))
T_funcs[0, 0].parameters.items()

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive_output
from matplotlib import cm
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata

resonances_plot = sorted(
    reaction.get_intermediate_particles(),
    key=lambda p: p.mass,
)
fig, axs = plt.subplots(nrows=2, figsize=(7, 14), tight_layout=True)
ax, ax_3d = axs
x_min, x_max = 0.5, 1.2
y_min, y_max = -1, 1
x_values = np.linspace(x_min, x_max, num=160)
y_values = np.linspace(y_min, y_max, num=100)
X, Y = np.meshgrid(x_values, y_values)
x = np.linspace(0.8, 1.5, num=500)
data1 = {"m_01": x}
data2 = {"m_01": X + Y * 1j}

LINES: list[Line2D] | None = None
color_mesh = None

selected_threshs = []
for i in range(n_channels):
    channel_count = +1
    selected_threshs.append(
        T_funcs[channel_count, i].parameters[f"m_{{0,{i}}}"]
        + T_funcs[channel_count, i].parameters[f"m_{{1,{i}}}"]
    )


def set_parameters(func, parameters):
    filtered_parameters = {
        k: v for k, v in parameters.items() if k in func.parameters
    }
    func.update_parameters(filtered_parameters)


def plot(*, i, j, **parameters):
    global LINES
    global color_mesh
    ax.set_title(f"T_func[{i}, {j}]")
    ax.set_xlabel("m [GeV]")
    ax.set_ylabel(Rf"$T_{{{i}{j}}}$")

    ax_3d.set_title(f"T_func[{i}, {j}]")
    ax_3d.set_xlabel("Re s")
    ax_3d.set_ylabel("Im s")
    # ax_3d.set_zlabel("Amplitude")

    set_parameters(T_funcs[i, j], parameters)
    y = T_funcs[i, j](data1)
    y_real = y.real
    y_imag = y.imag
    y_abs = np.abs(y)

    Z = T_funcs[i, j](data2)
    Z_values_imag = Z.imag
    Z_values_real = Z.real

    resonance_masses = {}
    for par, value in T_funcs[i, j].parameters.items():
        if par.startswith("m_{a"):
            resonance_masses[par] = value
    if LINES is None:
        LINES = [
            ax.plot(
                x,
                y_real,
                c="magenta",
                lw=2,
                label=R"Real",
            )[0],
            ax.plot(
                x,
                y_abs,
                c="red",
                lw=2,
                label=R"Abs",
            )[0],
            ax.plot(
                x,
                y_imag,
                c="blue",
                lw=2,
                label=R"Imag",
            )[0],
        ]
        res_count = -1
        thresh_count = -1
        for res, value in resonance_masses.items():
            res_count += 1
            LINES.append(
                ax.axvline(
                    value,
                    linestyle="dashed",
                    color=f"C{res_count}",
                    label=Rf"${res}$",
                ),
            )
        for thr in selected_threshs:
            thresh_count += 1
            LINES.append(
                ax.axvline(
                    thr,
                    linestyle="dashed",
                    color=f"C{thresh_count+1}",
                    label=Rf"Threshhold {thresh_count}",
                ),
            )
        ax.legend()
    else:
        LINES[0].set_data(x, y_real)
        LINES[1].set_data(x, y_abs)
        LINES[2].set_data(x, y_imag)
        for k, (res, value) in enumerate(resonance_masses.items(), 3):
            LINES[k].set_data([value, value], [0, 1])
        for thr in selected_threshs:
            LINES[4].set_data([thr, thr], [0, 1])
    if color_mesh is None:
        color_mesh = ax_3d.pcolormesh(X, Y, Z_values_imag, cmap=cm.coolwarm)
    else:
        color_mesh.set_array(Z_values_imag)
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw()


UI = VBox([slider_group])
output = interactive_output(plot, controls=sliders)
display(VBox([UI, output]))